# **Pipeline de Prétraitement et Tokenisation pour la Détection de Sarcasme**

**Importation des bibliothèques et préparation de l’environnement**

In [ ]:
import numpy as np
import pandas as pd
import re
from collections import Counter
import json

**CHARGEMENT DES DONNÉES**

In [ ]:
# 1. CHARGEMENT DES DONNÉES
# Charger le dataset de sarcasme
# Format attendu: JSON avec 'headline' et 'is_sarcastic'
with open('/content/Sarcasm_Headlines_Dataset.json', 'r') as f:
    data = [json.loads(line) for line in f]

df = pd.DataFrame(data)
print(f"Nombre total d'exemples: {len(df)}")
print(f"Distribution: {df['is_sarcastic'].value_counts()}")

Nombre total d'exemples: 26709
Distribution: is_sarcastic
0    14985
1    11724
Name: count, dtype: int64


**PRÉTRAITEMENT DES TEXTES**

In [ ]:
# 2. PRÉTRAITEMENT DES TEXTES
def pretraiter_texte(texte):
    """
    Nettoie le texte:
    - Enlever les URLs
    - Enlever les majuscules (convertir en minuscules)
    - Garder seulement les caractères alphabétiques et espaces
    """
    # Enlever les URLs
    texte = re.sub(r'http\S+|www\S+|https\S+', '', texte, flags=re.MULTILINE)
    # Convertir en minuscules
    texte = texte.lower()
    # Enlever la ponctuation et garder seulement lettres et espaces
    texte = re.sub(r'[^a-z\s]', '', texte)
    # Enlever les espaces multiples
    texte = re.sub(r'\s+', ' ', texte).strip()
    return texte

# Appliquer le prétraitement
df['texte_propre'] = df['headline'].apply(pretraiter_texte)
print("\nExemple de prétraitement:")
print(f"Original: {df['headline'].iloc[0]}")
print(f"Propre: {df['texte_propre'].iloc[0]}")



Exemple de prétraitement:
Original: former versace store clerk sues over secret 'black code' for minority shoppers
Propre: former versace store clerk sues over secret black code for minority shoppers


**TOKENIZATION**

In [ ]:
# 3. TOKENIZATION
def tokeniser(texte):
    """Sépare le texte en mots (tokens)"""
    return texte.split()

# Tokeniser tous les textes
df['tokens'] = df['texte_propre'].apply(tokeniser)


**CRÉATION DU VOCABULAIRE**

In [ ]:
# 4. CRÉATION DU VOCABULAIRE
def creer_vocabulaire(liste_tokens, taille_max=5000):
    """
    Crée un vocabulaire à partir des tokens
    taille_max: nombre maximum de mots dans le vocabulaire
    """
    # Compter tous les mots
    tous_mots = []
    for tokens in liste_tokens:
        tous_mots.extend(tokens)

    # Obtenir les mots les plus fréquents
    compteur = Counter(tous_mots)
    mots_frequents = compteur.most_common(taille_max)

    # Créer le vocabulaire (mot -> index)
    vocabulaire = {'<PAD>': 0, '<UNK>': 1}  # Tokens spéciaux
    for idx, (mot, _) in enumerate(mots_frequents, start=2):
        vocabulaire[mot] = idx

    return vocabulaire

vocabulaire = creer_vocabulaire(df['tokens'], taille_max=5000)
print(f"\nTaille du vocabulaire: {len(vocabulaire)}")
print(f"Exemples de mots: {list(vocabulaire.keys())[:10]}")



Taille du vocabulaire: 5002
Exemples de mots: ['<PAD>', '<UNK>', 'to', 'of', 'the', 'in', 'for', 'a', 'on', 'and']


**CONVERSION DES TOKENS EN INDICES**

In [ ]:
# 5. CONVERSION DES TOKENS EN INDICES
def tokens_vers_indices(tokens, vocab, max_len=20):
    """
    Convertit une liste de tokens en indices
    max_len: longueur maximale de la séquence
    """
    indices = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    # Padding ou troncature
    if len(indices) < max_len:
        indices += [vocab['<PAD>']] * (max_len - len(indices))
    else:
        indices = indices[:max_len]
    return indices

MAX_LENGTH = 20
df['indices'] = df['tokens'].apply(lambda x: tokens_vers_indices(x, vocabulaire, MAX_LENGTH))


**PRÉPARATION DES DONNÉES POUR L'ENTRAÎNEMENT**

In [ ]:
# 6. PRÉPARATION DES DONNÉES POUR L'ENTRAÎNEMENT
X = np.array(df['indices'].tolist())
y = np.array(df['is_sarcastic'])

# Division train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nFormes des données:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")


Formes des données:
X_train: (21367, 20)
y_train: (21367,)


**COUCHE D'EMBEDDING**

In [ ]:
# 7. COUCHE D'EMBEDDING
class Embedding:
    """Couche d'embedding simple"""
    def __init__(self, taille_vocab, dim_embedding):
        self.taille_vocab = taille_vocab
        self.dim_embedding = dim_embedding
        # Initialisation aléatoire des embeddings
        self.W = np.random.randn(taille_vocab, dim_embedding) * 0.01

    def forward(self, indices):
        """
        indices: (batch_size, max_length)
        retourne: (batch_size, max_length, dim_embedding)
        """
        return self.W[indices]

    def backward(self, grad_output, indices):
        """Calcul du gradient pour les embeddings"""
        grad_W = np.zeros_like(self.W)
        np.add.at(grad_W, indices.flatten(), grad_output.reshape(-1, self.dim_embedding))
        return grad_W


**FONCTION SOFTMAX**

In [ ]:
# 8. FONCTION SOFTMAX
def softmax(x):
    """Calcule la softmax de manière stable numériquement"""
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

**CROSS-ENTROPY LOSS**

In [ ]:
# 9. CROSS-ENTROPY LOSS
def cross_entropy_loss(y_pred, y_true):
    """
    y_pred: probabilités prédites (batch_size, 2)
    y_true: labels vrais (batch_size,)
    """
    batch_size = y_pred.shape[0]
    # Éviter log(0)
    y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
    # Loss pour chaque exemple
    correct_probs = y_pred_clipped[range(batch_size), y_true]
    loss = -np.mean(np.log(correct_probs))
    return loss

**MODÈLE SIMPLE DE CLASSIFICATION**

In [ ]:
# 10. MODÈLE SIMPLE DE CLASSIFICATION
class ClassifieurSarcasme:
    """Modèle simple: Embedding -> Moyenne -> Dense -> Softmax"""
    def __init__(self, taille_vocab, dim_embedding, num_classes=2):
        self.embedding = Embedding(taille_vocab, dim_embedding)
        # Couche dense pour la classification
        self.W = np.random.randn(dim_embedding, num_classes) * 0.01
        self.b = np.zeros(num_classes)

    def forward(self, X):
        """
        X: (batch_size, max_length)
        """
        # Embedding
        embedded = self.embedding.forward(X)  # (batch_size, max_length, dim_embedding)

        # Moyenne sur la longueur de séquence
        self.embedded_mean = np.mean(embedded, axis=1)  # (batch_size, dim_embedding)

        # Couche dense
        logits = np.dot(self.embedded_mean, self.W) + self.b  # (batch_size, num_classes)

        # Softmax
        probs = softmax(logits)

        return probs

    def backward(self, X, y_true, y_pred, learning_rate):
        """Rétropropagation et mise à jour des poids"""
        batch_size = X.shape[0]

        # Gradient de la softmax + cross-entropy
        grad_logits = y_pred.copy()
        grad_logits[range(batch_size), y_true] -= 1
        grad_logits /= batch_size

        # Gradients pour W et b
        grad_W = np.dot(self.embedded_mean.T, grad_logits)
        grad_b = np.sum(grad_logits, axis=0)

        # Mise à jour
        self.W -= learning_rate * grad_W
        self.b -= learning_rate * grad_b

    def predict(self, X):
        """Prédiction des classes"""
        probs = self.forward(X)
        return np.argmax(probs, axis=1)


**ENTRAÎNEMENT**

In [ ]:
# 11. ENTRAÎNEMENT
TAILLE_VOCAB = len(vocabulaire)
DIM_EMBEDDING = 50
LEARNING_RATE = 0.01
BATCH_SIZE = 32
EPOCHS = 10

modele = ClassifieurSarcasme(TAILLE_VOCAB, DIM_EMBEDDING)

print("\n=== ENTRAÎNEMENT ===")
for epoch in range(EPOCHS):
    # Mélanger les données
    indices = np.random.permutation(len(X_train))
    X_train_shuffled = X_train[indices]
    y_train_shuffled = y_train[indices]

    total_loss = 0
    num_batches = len(X_train) // BATCH_SIZE

    for i in range(num_batches):
        # Batch
        start_idx = i * BATCH_SIZE
        end_idx = start_idx + BATCH_SIZE
        X_batch = X_train_shuffled[start_idx:end_idx]
        y_batch = y_train_shuffled[start_idx:end_idx]

        # Forward pass
        y_pred = modele.forward(X_batch)

        # Calcul de la loss
        loss = cross_entropy_loss(y_pred, y_batch)
        total_loss += loss

        # Backward pass
        modele.backward(X_batch, y_batch, y_pred, LEARNING_RATE)

    # Évaluation sur le test
    y_pred_test = modele.predict(X_test)
    accuracy_test = np.mean(y_pred_test == y_test)

    avg_loss = total_loss / num_batches
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {avg_loss:.4f} - Test Accuracy: {accuracy_test:.4f}")



=== ENTRAÎNEMENT ===
Epoch 1/10 - Loss: 0.6868 - Test Accuracy: 0.5608
Epoch 2/10 - Loss: 0.6857 - Test Accuracy: 0.5608
Epoch 3/10 - Loss: 0.6857 - Test Accuracy: 0.5608
Epoch 4/10 - Loss: 0.6857 - Test Accuracy: 0.5608
Epoch 5/10 - Loss: 0.6857 - Test Accuracy: 0.5608
Epoch 6/10 - Loss: 0.6857 - Test Accuracy: 0.5608
Epoch 7/10 - Loss: 0.6857 - Test Accuracy: 0.5608
Epoch 8/10 - Loss: 0.6856 - Test Accuracy: 0.5608
Epoch 9/10 - Loss: 0.6857 - Test Accuracy: 0.5608
Epoch 10/10 - Loss: 0.6857 - Test Accuracy: 0.5608


**ÉVALUATION FINALE**

In [ ]:
# 12. ÉVALUATION FINALE
print("\n=== ÉVALUATION FINALE ===")
y_pred_train = modele.predict(X_train)
y_pred_test = modele.predict(X_test)

accuracy_train = np.mean(y_pred_train == y_train)
accuracy_test = np.mean(y_pred_test == y_test)

print(f"Précision sur l'entraînement: {accuracy_train:.4f}")
print(f"Précision sur le test: {accuracy_test:.4f}")


=== ÉVALUATION FINALE ===
Précision sur l'entraînement: 0.5611
Précision sur le test: 0.5608


**TEST SUR DE NOUVEAUX EXEMPLES**

In [ ]:
# 13. TEST SUR DE NOUVEAUX EXEMPLES
def predire_sarcasme(texte, modele, vocab):
    """Prédit si un texte est sarcastique"""
    # Prétraitement
    texte_propre = pretraiter_texte(texte)
    tokens = tokeniser(texte_propre)
    indices = tokens_vers_indices(tokens, vocab, MAX_LENGTH)

    # Prédiction
    X_input = np.array([indices])
    probs = modele.forward(X_input)
    classe = np.argmax(probs[0])

    return "Sarcastique" if classe == 1 else "Non-sarcastique", probs[0]


**Exemples de test**

In [ ]:
# Exemples de test
exemples = [
    "Nation Suddenly Realizes This Just Going To Be A Thing That Happens From Now On",
    "Scientists Discover New Species of Rare Butterfly in Amazon Rainforest"
]

print("\n=== PRÉDICTIONS SUR NOUVEAUX EXEMPLES ===")
for exemple in exemples:
    prediction, probs = predire_sarcasme(exemple, modele, vocabulaire)
    print(f"\nTexte: {exemple}")
    print(f"Prédiction: {prediction}")
    print(f"Probabilités: Non-sarcastique={probs[0]:.3f}, Sarcastique={probs[1]:.3f}")


=== PRÉDICTIONS SUR NOUVEAUX EXEMPLES ===

Texte: Nation Suddenly Realizes This Just Going To Be A Thing That Happens From Now On
Prédiction: Non-sarcastique
Probabilités: Non-sarcastique=0.562, Sarcastique=0.438

Texte: Scientists Discover New Species of Rare Butterfly in Amazon Rainforest
Prédiction: Non-sarcastique
Probabilités: Non-sarcastique=0.562, Sarcastique=0.438
